In [ ]:
!pip install langchain langchain_groq langchain_community langchain-huggingface langchain-pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.18
    Uninstalling pydantic-1.10.18:
      S

In [ ]:
!pip install -qU \
    openai==0.27.7 \
    pinecone-client==3.1.0 \
    pinecone-datasets==0.7.0 \
    tqdm \
    pinecone-notebooks==0.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 77.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.15 requires p

In [ ]:
import json
from datetime import datetime
from typing import Dict, List

from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.schema import Document



def format_datetime(date_str: str) :

    dt = datetime.fromisoformat(date_str.replace('Z', '+00:00'))
    return dt.strftime("%Y-%m-%d %H:%M")

def create_documents(booking_data: Dict) :

    documents = []
    user_id = booking_data['user']['id']

    for flight in booking_data['user']['flights']:
        # Main flight details document
        flight_text = (
            f"Flight Booking Details:\n"
            f"PNR: {flight['pnr']}\n"
            f"Class: {flight['class']}\n"
            f"From: {flight['source']}\n"
            f"To: {flight['destination']}\n"
            f"Departure: {format_datetime(flight['departure_date'])}\n"
            f"Arrival: {format_datetime(flight['arrival_date'])}\n"
            f"Layover Duration: {flight['layover_duration']}"
        )

        documents.append(Document(
            page_content=flight_text,
            metadata={
                'type': 'flight',
                'user_id': user_id,
                'ticket_id': flight['ticket_id'],
                'pnr': flight['pnr']
            }
        ))

        # Process each flight segment
        for idx, segment in enumerate(flight['segments']):
            segment_text = (
                f"Flight Segment {idx + 1}:\n"
                f"Flight Number: {segment['flight_number']}\n"
                f"From: {segment['departure']['airport']} ({segment['departure']['iata']})\n"
                f"To: {segment['arrival']['airport']} ({segment['arrival']['iata']})\n"
                f"Departure: {format_datetime(segment['departure']['date'])}\n"
                f"Arrival: {format_datetime(segment['arrival']['date'])}\n\n"
                f"Passenger Details for this segment:"
            )

            # Add passenger details
            for passenger in segment['passengers']:
                segment_text += f"\n- {passenger['first_name']} {passenger['last_name']}:"
                segment_text += f"\n  Seat: {passenger['seat_number']}"
                segment_text += f"\n  Cabin Baggage: {passenger['cabin_baggage']}"
                segment_text += f"\n  Check-in Baggage: {passenger['check_in_baggage']}"

            documents.append(Document(
                page_content=segment_text,
                metadata={
                    'type': 'segment',
                    'user_id': user_id,
                    'ticket_id': flight['ticket_id'],
                    'pnr': flight['pnr'],
                    'flight_number': segment['flight_number'],
                    'segment_number': idx
                }
            ))

    return documents


In [ ]:
import os

if not os.environ.get("PINECONE_API_KEY"):
    from pinecone_notebooks.colab import Authenticate
    Authenticate()

In [ ]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore


api_key = os.environ.get("PINECONE_API_KEY")

# configure client
pc = Pinecone(api_key=api_key,embeddings=embeddings)

In [ ]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [ ]:
index_name = 'havahavai1'

In [ ]:
# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=384,  # dimensionality of text-embedding-ada-002
        metric='cosine',
        spec=spec
    )
# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
with open("/content/Journey_Details.json", 'r') as f:
        booking_data = json.load(f)

documents = create_documents(booking_data)

In [ ]:
print(documents)

[Document(metadata={'type': 'flight', 'user_id': 227, 'ticket_id': 3183, 'pnr': 'HZAVJJ'}, page_content='Flight Booking Details:\nPNR: HZAVJJ\nClass: ECONOMY\nFrom: Cape Town International Airport (CPT)\nTo: Indira Gandhi International Airport (DEL)\nDeparture: 2024-07-11 14:35\nArrival: 2024-07-12 08:10\nLayover Duration: 55m'), Document(metadata={'type': 'segment', 'user_id': 227, 'ticket_id': 3183, 'pnr': 'HZAVJJ', 'flight_number': 'ET846', 'segment_number': 0}, page_content='Flight Segment 1:\nFlight Number: ET846\nFrom: Cape Town International Airport (CPT)\nTo: Addis Ababa Bole International Airport (ADD)\nDeparture: 2024-07-11 14:35\nArrival: 2024-07-11 22:00\n\nPassenger Details for this segment:\n- surendra singh:\n  Seat: 21a\n  Cabin Baggage: 7kg\n  Check-in Baggage: 23kg\n- narinder kaur:\n  Seat: 21b\n  Cabin Baggage: 7kg\n  Check-in Baggage: 23kg\n- samik singh:\n  Seat: 21c\n  Cabin Baggage: 7kg\n  Check-in Baggage: 23kg'), Document(metadata={'type': 'segment', 'user_id'

In [ ]:
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

In [ ]:
# documents = create_documents(booking_data)
vectorstore.add_documents(documents)

['f2d5832a-5a44-419e-a112-fd2f477949e2',
 'db355bcb-4de4-4f4b-ba2a-bc5b402e9a2d',
 '0bad9075-85df-4f4d-8bb1-d97d50a71a56']

## Retrieval

In [ ]:
from langchain_groq import ChatGroq

# get api key from platform.openai.com
groq_api_key = os.getenv('GROQ_API_KEY') or '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'



In [ ]:
query = (
    "What time is my flight from Cape Town to Addis Ababa, and what's the arrival time? "
)

res = embeddings.embed_query(query)
sim=vectorstore.similarity_search(query)

print(sim)

[Document(id='db355bcb-4de4-4f4b-ba2a-bc5b402e9a2d', metadata={'flight_number': 'ET846', 'pnr': 'HZAVJJ', 'segment_number': 0.0, 'ticket_id': 3183.0, 'type': 'segment', 'user_id': 227.0}, page_content='Flight Segment 1:\nFlight Number: ET846\nFrom: Cape Town International Airport (CPT)\nTo: Addis Ababa Bole International Airport (ADD)\nDeparture: 2024-07-11 14:35\nArrival: 2024-07-11 22:00\n\nPassenger Details for this segment:\n- surendra singh:\n  Seat: 21a\n  Cabin Baggage: 7kg\n  Check-in Baggage: 23kg\n- narinder kaur:\n  Seat: 21b\n  Cabin Baggage: 7kg\n  Check-in Baggage: 23kg\n- samik singh:\n  Seat: 21c\n  Cabin Baggage: 7kg\n  Check-in Baggage: 23kg'), Document(id='f2d5832a-5a44-419e-a112-fd2f477949e2', metadata={'pnr': 'HZAVJJ', 'ticket_id': 3183.0, 'type': 'flight', 'user_id': 227.0}, page_content='Flight Booking Details:\nPNR: HZAVJJ\nClass: ECONOMY\nFrom: Cape Town International Airport (CPT)\nTo: Indira Gandhi International Airport (DEL)\nDeparture: 2024-07-11 14:35\nArr

We write some functions to handle the retrieval and completion steps:

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.memory import ConversationBufferMemory

from langchain.prompts import ChatPromptTemplate

In [ ]:

def create_qa_chain(vectorstorePineconeVectorStore,groq_api_key):
    """Create the QA chain with custom prompt"""
    # Initialize language model
    llm = ChatGroq(
        api_key=groq_api_key,
        temperature=0,
        model_name="llama-3.1-8b-instant"
    )

    # Create conversation memory
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True
    )

    # Define custom prompt template
    prompt_template = """You are a helpful flight booking assistant. Use the following pieces of context to answer the user's question.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Always maintain a professional and friendly tone.
    Format dates and times in a readable way.
    If multiple passengers are involved, list their details clearly.
    For questions about baggage, specify both cabin and check-in allowances.
    For layover questions, include both the location and duration.

    Context: {context}

    Chat History: {chat_history}

    Question: {question}

    Assistant: """

    PROMPT = ChatPromptTemplate.from_template(prompt_template)

    return ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 1}
        ),
        memory=memory,
        combine_docs_chain_kwargs={"prompt": PROMPT}
    )

In [ ]:
qa_chain = create_qa_chain(vectorstore, groq_api_key)

In [ ]:
def get_chatbot_response(question: str, qa_chain: ConversationalRetrievalChain) -> str:
    """Get response from the chatbot for a given question"""
    response = qa_chain.invoke({"question": question})
    print(vectorstore.similarity_search(question))
    return response['answer']

In [ ]:
questions = [
    "What time do I arrive in Delhi?",
    "Where is my layover?",
    "What’s my seat for the first flight?",
    "Do I have checked baggage?"
]

for question in questions:
    response = get_chatbot_response(question, qa_chain)
    print(f"\nQ: {question}")
    print(f"A: {response}\n")

[Document(id='f2d5832a-5a44-419e-a112-fd2f477949e2', metadata={'pnr': 'HZAVJJ', 'ticket_id': 3183.0, 'type': 'flight', 'user_id': 227.0}, page_content='Flight Booking Details:\nPNR: HZAVJJ\nClass: ECONOMY\nFrom: Cape Town International Airport (CPT)\nTo: Indira Gandhi International Airport (DEL)\nDeparture: 2024-07-11 14:35\nArrival: 2024-07-12 08:10\nLayover Duration: 55m'), Document(id='0bad9075-85df-4f4d-8bb1-d97d50a71a56', metadata={'flight_number': 'ET686', 'pnr': 'HZAVJJ', 'segment_number': 1.0, 'ticket_id': 3183.0, 'type': 'segment', 'user_id': 227.0}, page_content='Flight Segment 2:\nFlight Number: ET686\nFrom: Addis Ababa Bole International Airport (ADD)\nTo: Indira Gandhi International Airport (DEL)\nDeparture: 2024-07-11 22:55\nArrival: 2024-07-12 08:10\n\nPassenger Details for this segment:\n- surendra singh:\n  Seat: 21a\n  Cabin Baggage: 7kg\n  Check-in Baggage: 23kg\n- narinder kaur:\n  Seat: 21b\n  Cabin Baggage: 7kg\n  Check-in Baggage: 23kg\n- samik singh:\n  Seat: 21

---